<a href="https://colab.research.google.com/github/dretcm/World-Cup-Challenge-2022-ACM/blob/main/worlds_cup_2k22_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load and preproccesing data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

results = pd.read_csv("InternationalFootballResults_1872to2022_results.csv")
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4.0,2.0,Friendly,London,England,False
2,1874-03-07,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2.0,2.0,Friendly,London,England,False
4,1876-03-04,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,False


In [4]:
def onehotencoder(countrys, word):
  output = []
  for country in countrys:
    if country == word:
      output.append(1)
    else:
      output.append(0)
  return output

countrys = list(set(list(results['home_team']) + list(results['away_team'])))
len(onehotencoder(countrys, 'Argentina'))

316

In [5]:
home = list(results['home_team'])
home_score = list(results['home_score'])

away = list(results['away_team'])
away_score = list(results['away_score'])

winners = [home[i] if home_score[i]>away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
losers = [home[i] if home_score[i]<away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
ties = [home[i] for i in range(len(home)) if home_score[i]==away_score[i]]
ties += [away[i] for i in range(len(away)) if home_score[i]==away_score[i]]

table_winners = {}
for c in countrys:
    table_winners[c] = winners.count(c)

table_losers = {}
for c in countrys:
    table_losers[c] = losers.count(c)

table_ties = {}
for c in countrys:
    table_ties[c] = ties.count(c)

table_plays = {}
for c in countrys:
  table_plays[c] = table_winners[c] + table_losers[c] + table_ties[c]


table_result = []   # 0(tie), 1(win team 1), 2(win team 2)
for i in range(len(home_score)):
  if home_score[i] == away_score[i]:
    table_result.append(0)
  elif home_score[i] > away_score[i]:
    table_result.append(1)
  else:
    table_result.append(2)


'''
for i in sorted(table, key= lambda x: table[x], reverse=True):
  print(i + ' win: ', table[i])
'''


def get_partys_teams(t1,t2):
  first = results[results['home_team']==t1]
  second = first[first['away_team']==t2]
  third = results[results['home_team']==t2]
  fourth = third[third['away_team']==t1]

  a = second[second['home_score']>second['away_score']].shape[0]
  a += fourth[fourth['home_score']<fourth['away_score']].shape[0]

  b = second[second['home_score']<second['away_score']].shape[0]
  b += fourth[fourth['home_score']>fourth['away_score']].shape[0]

  c = second[second['home_score']==second['away_score']].shape[0]
  c += fourth[fourth['home_score']==fourth['away_score']].shape[0]
  return [a,b,c]

get_partys_teams("Spain", "Brazil"), get_partys_teams("Brazil", "Spain")

([2, 5, 2], [5, 2, 2])

In [6]:
import json
# table_1_vs_2 = {c+"_"+c2:get_partys_teams(c,c2) for c in countrys for c2 in countrys if c!=c2}
# with open("data_1vs2.json", "w") as write_file:
#     json.dump(table_1_vs_2, write_file)

with open("data_1vs2.json", "r") as read_file:
    table_1_vs_2 = json.load(read_file)

In [ ]:
table_1_vs_2['Spain_Brazil'], table_1_vs_2['Brazil_Spain']

([2, 5, 2], [5, 2, 2])

In [ ]:
np.mean(list(table_plays.values())), np.std(list(table_plays.values()))

(278.8607594936709, 270.4569496698867)

In [ ]:
max(table_plays.values()), len(home)

(1051, 44060)

In [7]:
def normalize(table, standar=True):
  mean = np.mean(list(table.values())) if standar else 0
  std =  np.std(list(table.values())) if standar else max(table.values())
  return mean, std

mean, std = normalize(table_plays)
played1 = [(table_plays[country]-mean)/std for country in home] 
played2 =  [(table_plays[country]-mean)/std for country in away]

mean, std = normalize(table_winners)
win1 =  [(table_winners[country]-mean)/std for country in home]
win2 = [(table_winners[country]-mean)/std for country in away]

mean, std = normalize(table_losers)
lose1 = [(table_losers[country]-mean)/std for country in home]
lose2 = [(table_losers[country]-mean)/std for country in away]


arr = []
for i in range(len(home)):
  arr.append(onehotencoder(countrys, home[i]) + onehotencoder(countrys, away[i]))
arr = np.array(arr)

titles = [country+' '+str(i) for i in range(1,3) for country in countrys]
onehot_countrys = pd.DataFrame(arr, columns=titles)

results['played 1'] = played1
results['played 2'] = played2
results['win 1'] = win1
results['win 2'] = win2
results['lose 1'] = lose1
results['lose 2'] = lose2

results['win_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][0], axis=1)
results['lose_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['away_team']+"_"+x['home_team']][0], axis=1)
results['tie_1_vs_2'] = results.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][2], axis=1)
results['total_1_vs_2'] = results.apply(lambda x: x['win_1_vs_2']+ x['lose_1_vs_2'] + x['tie_1_vs_2'], axis=1)

#max_win_vs = max(results['win_1_vs_2'])
data_1_normalize = {}
data_1_normalize['win_1_vs_2']= [np.mean(list(results['win_1_vs_2'])), np.std(list(results['win_1_vs_2']))]
results['win_1_vs_2'] = results['win_1_vs_2'].map(lambda x: (x-data_1_normalize['win_1_vs_2'][0])/data_1_normalize['win_1_vs_2'][1])

#max_lose_vs = max(results['lose_1_vs_2'])
data_1_normalize['lose_1_vs_2']= [np.mean(list(results['lose_1_vs_2'])), np.std(list(results['lose_1_vs_2']))]
results['lose_1_vs_2'] = results['lose_1_vs_2'].map(lambda x: (x-data_1_normalize['lose_1_vs_2'][0])/data_1_normalize['lose_1_vs_2'][1])

#max_total_vs = max(results['total_1_vs_2'])
data_1_normalize['total_1_vs_2']= [np.mean(list(results['total_1_vs_2'])), np.std(list(results['total_1_vs_2']))]
results['total_1_vs_2'] = results['total_1_vs_2'].map(lambda x: (x-data_1_normalize['total_1_vs_2'][0])/data_1_normalize['total_1_vs_2'][1])

results['result'] = table_result
#results['difference'] = results.apply(lambda x: x['home_score'] - x['away_score'], axis=1)

results['neutral'] = results['neutral'].map(lambda x: 1 if x==True else 0)

results = pd.concat([results, onehot_countrys], axis=1)

results['date'] = results['date'].map(lambda x: (int(str(x)[:4])))
date_mean, date_std = np.mean(list(results['date'])), np.std(list(results['date']))
results['date'] = results['date'].map(lambda x: (x-date_mean)/date_std)

# model 1:
data_1 = results.copy()
data_1.where(data_1['home_score']!=data_1['away_score'] ,inplace=True)
data_1 = data_1.drop(['city', 'country', 'tournament', 'home_score','away_score', 'home_team', 
                      'away_team','tie_1_vs_2'], axis=1)
data_1 = data_1.dropna(axis=0)
data_1

,date,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,win_1_vs_2,lose_1_vs_2,...,Székely Land 2,Libya 2,Western Australia 2,Kosovo 2,Cameroon 2,Cuba 2,Brittany 2,Andalusia 2,Yorkshire 2,Philippines 2
1,-4.832257,0.0,2.829061,1.974951,3.813250,2.183598,1.012904,1.570149,3.387917,2.971062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-4.791581,0.0,1.974951,2.829061,2.183598,3.813250,1.570149,1.012904,2.871342,3.498809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-4.710229,0.0,1.974951,2.829061,2.183598,3.813250,1.570149,1.012904,2.871342,3.498809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-4.710229,0.0,1.974951,1.516468,2.183598,0.890844,1.570149,2.291291,4.507161,1.299862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-4.669553,0.0,2.829061,1.974951,3.813250,2.183598,1.012904,1.570149,3.387917,2.971062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44052,1.228444,0.0,1.165210,-0.158475,0.937854,-0.378404,0.958272,0.226204,-0.486392,-0.635211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44055,1.228444,0.0,2.045203,0.096648,1.564643,0.248385,2.498892,-0.221778,-0.572488,-0.459295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44057,1.228444,0.0,-0.775949,0.410931,-0.636955,-0.268716,-0.844582,1.570149,-0.572488,-0.723169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44058,1.228444,0.0,1.157815,1.457309,0.898679,0.506936,1.253284,2.608156,-0.228105,-0.459295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,played 1,...,Székely Land 2,Libya 2,Western Australia 2,Kosovo 2,Cameroon 2,Cuba 2,Brittany 2,Andalusia 2,Yorkshire 2,Philippines 2
0,-4.872932,Scotland,England,0.0,0.0,Friendly,Glasgow,Scotland,0,1.974951,...,0,0,0,0,0,0,0,0,0,0
1,-4.832257,England,Scotland,4.0,2.0,Friendly,London,England,0,2.829061,...,0,0,0,0,0,0,0,0,0,0
2,-4.791581,Scotland,England,2.0,1.0,Friendly,Glasgow,Scotland,0,1.974951,...,0,0,0,0,0,0,0,0,0,0
3,-4.750905,England,Scotland,2.0,2.0,Friendly,London,England,0,2.829061,...,0,0,0,0,0,0,0,0,0,0
4,-4.710229,Scotland,England,3.0,0.0,Friendly,Glasgow,Scotland,0,1.974951,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44055,1.228444,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,0,2.045203,...,0,0,0,0,0,0,0,0,0,0
44056,1.228444,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,0,2.854943,...,0,0,0,0,0,0,0,0,0,0
44057,1.228444,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,0,-0.775949,...,0,0,0,0,0,0,0,0,0,0
44058,1.228444,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,0,1.157815,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# Data for model 1:
Y = data_1['result']
X = data_1.drop(['result'], axis=1)
X

,date,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,win_1_vs_2,lose_1_vs_2,...,Székely Land 2,Libya 2,Western Australia 2,Kosovo 2,Cameroon 2,Cuba 2,Brittany 2,Andalusia 2,Yorkshire 2,Philippines 2
1,-4.832257,0.0,2.829061,1.974951,3.813250,2.183598,1.012904,1.570149,3.387917,2.971062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-4.791581,0.0,1.974951,2.829061,2.183598,3.813250,1.570149,1.012904,2.871342,3.498809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-4.710229,0.0,1.974951,2.829061,2.183598,3.813250,1.570149,1.012904,2.871342,3.498809,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,-4.710229,0.0,1.974951,1.516468,2.183598,0.890844,1.570149,2.291291,4.507161,1.299862,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,-4.669553,0.0,2.829061,1.974951,3.813250,2.183598,1.012904,1.570149,3.387917,2.971062,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44052,1.228444,0.0,1.165210,-0.158475,0.937854,-0.378404,0.958272,0.226204,-0.486392,-0.635211,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44055,1.228444,0.0,2.045203,0.096648,1.564643,0.248385,2.498892,-0.221778,-0.572488,-0.459295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44057,1.228444,0.0,-0.775949,0.410931,-0.636955,-0.268716,-0.844582,1.570149,-0.572488,-0.723169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44058,1.228444,0.0,1.157815,1.457309,0.898679,0.506936,1.253284,2.608156,-0.228105,-0.459295,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training NN-winner model

In [10]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
import keras

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(Y)
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [1]])

In [11]:
import random
import numpy as np
import tensorflow as tf
import os

seed = 10
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

num_classes = y_train.shape[1]
num_input = X.shape[1]

model_winner = Sequential()
model_winner.add(Dense(1024, input_dim=num_input, activation='relu'))
model_winner.add(Dense(2048, activation='relu'))
model_winner.add(Dense(1024, activation='relu'))
model_winner.add(Dense(512, activation='relu'))
model_winner.add(Dense(num_classes, activation='sigmoid'))

#model_winner.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_winner.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_winner.fit(X, y_train, epochs=200, batch_size=64)
model_winner.save('my_model_winner.h5')

Epoch 1/200
530/530 [==============================] - 5s 4ms/step - loss: 0.4895 - accuracy: 0.7641
Epoch 2/200
530/530 [==============================] - 2s 4ms/step - loss: 0.4485 - accuracy: 0.7879
Epoch 3/200
530/530 [==============================] - 2s 4ms/step - loss: 0.4269 - accuracy: 0.7974
Epoch 4/200
530/530 [==============================] - 4s 7ms/step - loss: 0.4105 - accuracy: 0.8034
Epoch 5/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3955 - accuracy: 0.8114
Epoch 6/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3785 - accuracy: 0.8162
Epoch 7/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3613 - accuracy: 0.8225
Epoch 8/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3474 - accuracy: 0.8278
Epoch 9/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3345 - accuracy: 0.8324
Epoch 10/200
530/530 [==============================] - 2s 4ms/step - loss: 0.3236 - accura

In [12]:
def convert_input_winner(country_1, country_2, paises=countrys, house='Qatar', year=2022):
  out = [(year-date_mean)/date_std] # date is 2022
  #out = []
  out.append(0 if country_1==house else 1)  # neutral: True/False : si jugarán en un lugar neutro, como es el mundial, todos juegan así.

  mean, std = normalize(table_plays)
  out.append((table_plays[country_1]-mean)/std)
  out.append((table_plays[country_2]-mean)/std)

  mean, std = normalize(table_winners)
  out.append((table_winners[country_1]-mean)/std)
  out.append((table_winners[country_2]-mean)/std)
  
  mean, std = normalize(table_losers)
  out.append((table_losers[country_1]-mean)/std)
  out.append((table_losers[country_2]-mean)/std)

  reply = table_1_vs_2[country_1+"_"+country_2]
  out.append((reply[0]-data_1_normalize['win_1_vs_2'][0])/data_1_normalize['win_1_vs_2'][1])
  out.append((reply[1]-data_1_normalize['lose_1_vs_2'][0])/data_1_normalize['lose_1_vs_2'][1])
  out.append((sum(reply)-data_1_normalize['total_1_vs_2'][0])/data_1_normalize['total_1_vs_2'][1])

  out += onehotencoder(paises,country_1) + onehotencoder(paises, country_2)

  return np.array([out])

test = convert_input_winner("Bolivia", "Brazil")
test[0][:25]

array([ 1.22844363,  1.        ,  0.77328107,  2.72553263,  0.01333911,
        4.25983767,  1.71219249,  0.57584825, -0.31420063,  1.29986219,
        0.36971639,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [13]:
model_winner = load_model('my_model_winner.h5')
test = convert_input_winner("Brazil", "Peru")
print(model_winner.predict(test))
test = convert_input_winner("Peru", "Brazil")
model_winner.predict(test)

1/1 [==============================] - 0s 122ms/step
[[0.39730105]]
1/1 [==============================] - 0s 19ms/step


array([[1.]], dtype=float32)

In [14]:
def predict_winner(country_1, country_2, model, tie = True, ver=False, year=2022):
  party_1 = model.predict(convert_input_winner(country_1, country_2, year=year), verbose=0)[0][0]
  party_2 = model.predict(convert_input_winner(country_2, country_1, year=year), verbose=0)[0][0]
  if ver:
    print(party_1, party_2)
  if (party_1<0.51) == (party_2<0.51):
    if tie:
      return 'Tie'
    else:
      if party_1>party_2:
        return country_2
      return country_1
  elif (party_1<0.51) and (party_2>0.51):
    return country_1
  return country_2

predict_winner("Brazil", "Peru", model_winner,False), predict_winner("Argentina", "Peru", model_winner, False, True), predict_winner("Peru","Argentina", model_winner, False, True)

0.26063648 1.0
1.0 0.26063648


('Brazil', 'Argentina', 'Argentina')

In [ ]:
test = pd.read_csv("/content/InternationalFootballResults_1872to2022_shootouts.csv")
c = list(test['home_team'])
c2 = list(test['away_team'])
w = list(test['winner'])
d = [int(i[:4]) for i in list(test['date'])]

replys = [True if predict_winner(c[i],c2[i], model_winner, tie=False, year=d[i])==w[i] else False for i in range(len(c))]
replys.count(True)/len(replys)

# Training NN-gols model

In [16]:
# data for model 2:
results = pd.read_csv("InternationalFootballResults_1872to2022_results.csv")

home = list(results['home_team'])
home_score = list(results['home_score'])

away = list(results['away_team'])
away_score = list(results['away_score'])

home,away = home+away, away+home
home_score,away_score = home_score+away_score, away_score+home_score

winners = [home[i] if home_score[i]>away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
losers = [home[i] if home_score[i]<away_score[i] else away[i] for i in range(len(home)) if home_score[i]!=away_score[i]]
ties = [home[i] for i in range(len(home)) if home_score[i]==away_score[i]]
ties += [away[i] for i in range(len(away)) if home_score[i]==away_score[i]]

table_winners = {}
for c in countrys:
    table_winners[c] = winners.count(c)//2

table_losers = {}
for c in countrys:
    table_losers[c] = losers.count(c)//2

table_ties = {}
for c in countrys:
    table_ties[c] = ties.count(c)//2

table_plays = {}
for c in countrys:
  table_plays[c] = table_winners[c] + table_losers[c] + table_ties[c]


table_result = []   # 0(tie), 1(win team 1), 2(win team 2)
for i in range(len(home_score)):
  if home_score[i] == away_score[i]:
    table_result.append(0)
  elif home_score[i] > away_score[i]:
    table_result.append(1)
  else:
    table_result.append(2)

mean, std = normalize(table_plays)
played1 = [(table_plays[country]-mean)/std for country in home]
played2 =  [(table_plays[country]-mean)/std for country in away]

mean, std = normalize(table_winners)
win1 =  [(table_winners[country]-mean)/std for country in home]
win2 = [(table_winners[country]-mean)/std for country in away]

mean, std = normalize(table_losers)
lose1 = [(table_losers[country]-mean)/std for country in home]
lose2 = [(table_losers[country]-mean)/std for country in away]

mean, std = normalize(table_ties)
tie1 = [(table_ties[country]-mean)/std for country in home]
tie2 = [(table_ties[country]-mean)/std for country in away]

arr = []
for i in range(len(home)):
  arr.append(onehotencoder(countrys, home[i]) + onehotencoder(countrys, away[i]))
arr = np.array(arr)

titles = [country+' '+str(i) for i in range(1,3) for country in countrys]
onehot_countrys = pd.DataFrame(arr, columns=titles)

results_2 = pd.concat([results, results], axis=0)

results_2['played 1'] = played1
results_2['played 2'] = played2
results_2['win 1'] = win1
results_2['win 2'] = win2
results_2['lose 1'] = lose1
results_2['lose 2'] = lose2
results_2['tie 1'] = tie1
results_2['tie 2'] = tie2


results_2['win_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][0], axis=1)
results_2['lose_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['away_team']+"_"+x['home_team']][0], axis=1)
results_2['tie_1_vs_2'] = results_2.apply(lambda x: table_1_vs_2[x['home_team']+"_"+x['away_team']][2], axis=1)
results_2['total_1_vs_2'] = results_2.apply(lambda x: x['win_1_vs_2']+ x['lose_1_vs_2'] + x['tie_1_vs_2'], axis=1)

data_2_normalize = {}
data_2_normalize['win_1_vs_2']= [np.mean(list(results_2['win_1_vs_2'])), np.std(list(results_2['win_1_vs_2']))]
results_2['win_1_vs_2'] = results_2['win_1_vs_2'].map(lambda x: (x-data_2_normalize['win_1_vs_2'][0])/data_2_normalize['win_1_vs_2'][1])

data_2_normalize['lose_1_vs_2']= [np.mean(list(results_2['lose_1_vs_2'])), np.std(list(results_2['lose_1_vs_2']))]
results_2['lose_1_vs_2'] = results_2['lose_1_vs_2'].map(lambda x: (x-data_2_normalize['lose_1_vs_2'][0])/data_2_normalize['lose_1_vs_2'][1])

data_2_normalize['tie_1_vs_2']= [np.mean(list(results_2['tie_1_vs_2'])), np.std(list(results_2['tie_1_vs_2']))]
results_2['tie_1_vs_2'] = results_2['tie_1_vs_2'].map(lambda x: (x-data_2_normalize['tie_1_vs_2'][0])/data_2_normalize['tie_1_vs_2'][1])

data_2_normalize['total_1_vs_2']= [np.mean(list(results_2['total_1_vs_2'])), np.std(list(results_2['total_1_vs_2']))]
results_2['total_1_vs_2'] = results_2['total_1_vs_2'].map(lambda x: (x-data_2_normalize['total_1_vs_2'][0])/data_2_normalize['total_1_vs_2'][1])

results_2['result'] = table_result
#results_2['difference'] = results_2.apply(lambda x: x['home_score'] - x['away_score'], axis=1)
results_2['gols team 1'] = home_score

results_2['neutral'] = results_2['neutral'].map(lambda x: 1 if x==True else 0)

results_2['date'] = results_2['date'].map(lambda x: (int(str(x)[:4])))
date_mean, date_std = np.mean(list(results_2['date'])), np.std(list(results_2['date']))
results_2['date'] = results_2['date'].map(lambda x: (x-date_mean)/date_std)

#print(results_2.shape, onehot_countrys.shape)
results_2.reset_index(inplace=True, drop=True)
onehot_countrys.reset_index(inplace=True, drop=True)
results_2 = pd.concat([results_2, onehot_countrys], axis=1)

data_2 = results_2.drop(['city', 'country', 'tournament','home_score', 'away_score', 'home_team', 'away_team'], axis=1)

data_2

,date,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,tie 1,tie 2,...,Székely Land 2,Libya 2,Western Australia 2,Kosovo 2,Cameroon 2,Cuba 2,Brittany 2,Andalusia 2,Yorkshire 2,Philippines 2
0,-4.872932,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,0,0,0
1,-4.832257,0,2.829060,1.974957,3.813252,2.183609,1.012924,1.570162,2.773155,1.668278,...,0,0,0,0,0,0,0,0,0,0
2,-4.791581,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,0,0,0
3,-4.750905,0,2.829060,1.974957,3.813252,2.183609,1.012924,1.570162,2.773155,1.668278,...,0,0,0,0,0,0,0,0,0,0
4,-4.710229,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88115,1.228444,0,0.096671,2.045208,0.248408,1.564658,-0.221740,2.498891,0.219993,1.862378,...,0,0,0,0,0,0,0,0,0,0
88116,1.228444,0,-0.014251,2.854942,-0.088489,3.202136,0.007711,2.192957,0.100546,2.429747,...,0,0,0,0,0,0,0,0,0,0
88117,1.228444,0,0.410951,-0.775919,-0.268690,-0.636926,1.570162,-0.844535,0.025893,-0.765438,...,0,0,0,1,0,0,0,0,0,0
88118,1.228444,0,1.457319,1.157829,0.506958,0.898699,2.608154,1.253301,1.354732,1.250216,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Y = data_2['gols team 1']
X = data_2.drop(['gols team 1','result'], axis=1)
X.insert(X.shape[1], "party tie", data_2['result'].map(lambda x: 0 if x!=0 else 1))
X.insert(X.shape[1], "win country 1", data_2['result'].map(lambda x: 0 if x!=1 else 1))
X.insert(X.shape[1], "win country 2", data_2['result'].map(lambda x: 0 if x!=2 else 1))
X.head()

,date,neutral,played 1,played 2,win 1,win 2,lose 1,lose 2,tie 1,tie 2,...,Kosovo 2,Cameroon 2,Cuba 2,Brittany 2,Andalusia 2,Yorkshire 2,Philippines 2,party tie,win country 1,win country 2
0,-4.872932,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,1,0,0
1,-4.832257,0,2.829060,1.974957,3.813252,2.183609,1.012924,1.570162,2.773155,1.668278,...,0,0,0,0,0,0,0,0,1,0
2,-4.791581,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,0,1,0
3,-4.750905,0,2.829060,1.974957,3.813252,2.183609,1.012924,1.570162,2.773155,1.668278,...,0,0,0,0,0,0,0,1,0,0
4,-4.710229,0,1.974957,2.829060,2.183609,3.813252,1.570162,1.012924,1.668278,2.773155,...,0,0,0,0,0,0,0,0,1,0


In [18]:
Y = Y.map(lambda x: abs(x))
Y

0        0.0
1        4.0
2        2.0
3        2.0
4        3.0
        ... 
88115    2.0
88116    1.0
88117    1.0
88118    1.0
88119    NaN
Name: gols team 1, Length: 88120, dtype: float64

In [19]:
Y = Y.fillna(0)
Y

0        0.0
1        4.0
2        2.0
3        2.0
4        3.0
        ... 
88115    2.0
88116    1.0
88117    1.0
88118    1.0
88119    0.0
Name: gols team 1, Length: 88120, dtype: float64

In [20]:
from keras.models import Sequential, load_model
from keras.layers import Dense
from sklearn.preprocessing import LabelBinarizer
import keras

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(Y)
y_train[:50]

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
y_train.shape

(88120, 26)

In [22]:
num_classes = y_train.shape[1]
num_input = X.shape[1]

model = Sequential()
model.add(Dense(1024, input_dim=num_input, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, y_train, epochs=100, batch_size=128)

Epoch 1/100
689/689 [==============================] - 3s 4ms/step - loss: 1.2893 - accuracy: 0.4411
Epoch 2/100
689/689 [==============================] - 3s 4ms/step - loss: 1.2446 - accuracy: 0.4523
Epoch 3/100
689/689 [==============================] - 3s 4ms/step - loss: 1.2286 - accuracy: 0.4589
Epoch 4/100
689/689 [==============================] - 3s 4ms/step - loss: 1.2105 - accuracy: 0.4643
Epoch 5/100
689/689 [==============================] - 3s 5ms/step - loss: 1.1890 - accuracy: 0.4743
Epoch 6/100
689/689 [==============================] - 3s 4ms/step - loss: 1.1596 - accuracy: 0.4871
Epoch 7/100
689/689 [==============================] - 3s 4ms/step - loss: 1.1197 - accuracy: 0.5049
Epoch 8/100
689/689 [==============================] - 3s 4ms/step - loss: 1.0712 - accuracy: 0.5274
Epoch 9/100
689/689 [==============================] - 3s 4ms/step - loss: 1.0174 - accuracy: 0.5508
Epoch 10/100
689/689 [==============================] - 3s 4ms/step - loss: 0.9591 - accura

In [53]:
model.fit(X, y_train, epochs=50, batch_size=128)

Epoch 1/50
689/689 [==============================] - 4s 6ms/step - loss: 0.2308 - accuracy: 0.8851
Epoch 2/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2229 - accuracy: 0.8883
Epoch 3/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2262 - accuracy: 0.8864
Epoch 4/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2323 - accuracy: 0.8860
Epoch 5/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2248 - accuracy: 0.8890
Epoch 6/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2171 - accuracy: 0.8902
Epoch 7/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2162 - accuracy: 0.8903
Epoch 8/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2207 - accuracy: 0.8900
Epoch 9/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2224 - accuracy: 0.8902
Epoch 10/50
689/689 [==============================] - 3s 4ms/step - loss: 0.2167 - accuracy: 0.8923

In [73]:
model.fit(X, y_train, epochs=50, batch_size=128)

Epoch 1/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1872 - accuracy: 0.9075
Epoch 2/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1867 - accuracy: 0.9062
Epoch 3/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1899 - accuracy: 0.9067
Epoch 4/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1952 - accuracy: 0.9059
Epoch 5/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1897 - accuracy: 0.9068
Epoch 6/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1920 - accuracy: 0.9067
Epoch 7/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1862 - accuracy: 0.9083
Epoch 8/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1806 - accuracy: 0.9101
Epoch 9/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1885 - accuracy: 0.9084
Epoch 10/50
689/689 [==============================] - 3s 4ms/step - loss: 0.1855 - accuracy: 0.9077

In [74]:
model.save('my_model_gols.h5')
clases = label_binarizer.classes_
clases

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 24., 30., 31.])

In [75]:
def convert_input_winner_2(country_1, country_2, paises=countrys, house='Qatar', year=2022):
  out = [(year-date_mean)/date_std] # date is 2022
  #out = []
  out.append(0 if country_1==house else 1)  # neutral: True/False : si jugarán en un lugar neutro, como es el mundial, todos juegan así.

  mean, std = normalize(table_plays)
  out.append((table_plays[country_1]-mean)/std)
  out.append((table_plays[country_2]-mean)/std)

  mean, std = normalize(table_winners)
  out.append((table_winners[country_1]-mean)/std)
  out.append((table_winners[country_2]-mean)/std)
  
  mean, std =normalize(table_losers)
  out.append((table_losers[country_1]-mean)/std)
  out.append((table_losers[country_2]-mean)/std)

  mean, std = normalize(table_ties)
  out.append((table_ties[country_1]-mean)/std)
  out.append((table_ties[country_2]-mean)/std)

  reply = table_1_vs_2[country_1+"_"+country_2]
  out.append((reply[0]-data_2_normalize['win_1_vs_2'][0])/data_2_normalize['win_1_vs_2'][1])
  out.append((reply[1]-data_2_normalize['lose_1_vs_2'][0])/data_2_normalize['lose_1_vs_2'][1])
  out.append((reply[2]-data_2_normalize['tie_1_vs_2'][0])/data_2_normalize['tie_1_vs_2'][1])
  out.append((sum(reply)-data_2_normalize['total_1_vs_2'][0])/data_2_normalize['total_1_vs_2'][1])

  out += onehotencoder(paises,country_1) + onehotencoder(paises, country_2)

  return np.array([out])

def predict_results_full(country_1, country_2, model_winner, model, clases, tie=True):
  test = list(convert_input_winner_2(country_1, country_2)[0])
  test_2 = list(convert_input_winner_2(country_2, country_1)[0])
  win = predict_winner(country_1, country_2, model_winner, tie)
  if win == 'Tie':
    win = country_1 + "_" + country_2
    test.extend([1,0,0])
    test_2.extend([1,0,0])
  elif win==country_1:
    win += "_"+country_2
    test.extend([0,1,0])
    test_2.extend([0,0,1])
  else:
    win += "_"+country_1
    test.extend([0,0,1])
    test_2.extend([0,1,0])

  test = np.array([test])
  test_2 = np.array([test_2])

  pred_1 = model.predict(test, verbose=0)[0]
  pred_2 = model.predict(test_2, verbose=0)[0]

  pos_1 = np.argmax(pred_1)
  pos_2 = np.argmax(pred_2)
  if not tie and pos_1==pos_2:
    if pred_1[pos_1]>pred_2[pos_2]:
      b = pred_1[pos_1]
      a = pred_1
      mx = -10000000
      for i in a:
        if i>mx and i!= b:
          mx = i
      for i in range(len(a)):
        if a[i]==mx:
          pos_1 = i
          break
    else:
      b = pred_2[pos_2]
      a = pred_2
      mx = -10000000
      for i in a:
        if i>mx and i!= b:
          mx = i
      for i in range(len(a)):
        if a[i]==mx:
          pos_2 = i
          break
  prediction = clases[pos_1]
  prediction_2 = clases[pos_2]
  return country_1 + " ("+str(int(prediction))+"-"+str(int(prediction_2))+") " + country_2

predict_results_full("Brazil", "Peru", model_winner, model, clases)

'Brazil (2-0) Peru'

In [76]:
predict_results_full("Peru", "Brazil", model_winner, model, clases), predict_results_full("Brazil", "Peru", model_winner, model, clases)

('Peru (0-2) Brazil', 'Brazil (2-0) Peru')

In [77]:
predict_results_full("Chile", "Peru", model_winner, model, clases), predict_results_full("Peru", "Chile", model_winner, model, clases)

('Chile (0-3) Peru', 'Peru (3-0) Chile')

In [78]:
predict_results_full("Qatar", "Peru", model_winner, model, clases), predict_results_full("Peru", "Qatar", model_winner, model, clases)

('Qatar (0-1) Peru', 'Peru (1-0) Qatar')

In [79]:
predict_results_full("Argentina", "Peru", model_winner, model, clases), predict_results_full("Peru","Argentina", model_winner, model, clases)

('Argentina (1-0) Peru', 'Peru (0-1) Argentina')

In [80]:
predict_results_full("Brazil", "Bolivia", model_winner, model, clases), predict_results_full("Bolivia", "Brazil", model_winner, model, clases)

('Brazil (3-0) Bolivia', 'Bolivia (0-3) Brazil')

In [81]:
def predict_results(country_1, country_2, model_winner, model, clases, tie=True):
  cad = predict_results_full(country_1, country_2, model_winner, model, clases, tie).split("-")
  team_1 = int(cad[0][-1])
  team_2 = int(cad[1][0])

  if team_1 > team_2:
    return cad[0][:len(cad[0])-3]+"_"+cad[1][3:]+"_+"+str(team_1-team_2)
  else:
    return cad[1][3:]+"_"+cad[0][:len(cad[0])-3]+"_+"+str(team_2-team_1)

predict_results("Brazil", "Bolivia", model_winner, model, clases)

'Brazil_Bolivia_+3'

In [82]:
predict_results("Chile", "Peru", model_winner, model, clases, False)

'Peru_Chile_+3'

# Grupos

In [83]:
Groups = {"Group A": ["Qatar",  "Ecuador", "Senegal" ,"Netherlands"],
          "Group B": ["England",  "Iran", "United States",  "Wales"],
          "Group C": ["Argentina",  "Saudi Arabia",  "Mexico",  "Poland"],
          "Group D": ["France",  "Australia",  "Denmark",  "Tunisia"],
          "Group E": ["Spain",  "Costa Rica",  "Germany",  "Japan"],
          "Group F": ["Belgium",  "Canada",  "Morocco", "Croatia"],
          "Group G": ["Brazil", "Serbia",  "Switzerland",  "Cameroon"],
          "Group H": ["Portugal",  "Ghana", "Uruguay",  "South Korea"]}
Groups

{'Group A': ['Qatar', 'Ecuador', 'Senegal', 'Netherlands'],
 'Group B': ['England', 'Iran', 'United States', 'Wales'],
 'Group C': ['Argentina', 'Saudi Arabia', 'Mexico', 'Poland'],
 'Group D': ['France', 'Australia', 'Denmark', 'Tunisia'],
 'Group E': ['Spain', 'Costa Rica', 'Germany', 'Japan'],
 'Group F': ['Belgium', 'Canada', 'Morocco', 'Croatia'],
 'Group G': ['Brazil', 'Serbia', 'Switzerland', 'Cameroon'],
 'Group H': ['Portugal', 'Ghana', 'Uruguay', 'South Korea']}

In [84]:
import itertools
list(itertools.combinations(Groups["Group A"], 2))

[('Qatar', 'Ecuador'),
 ('Qatar', 'Senegal'),
 ('Qatar', 'Netherlands'),
 ('Ecuador', 'Senegal'),
 ('Ecuador', 'Netherlands'),
 ('Senegal', 'Netherlands')]

In [85]:
def win_groups(group_X):
  print("======================>",group_X)
  partys = list(itertools.combinations(Groups[group_X], 2))
  match = []
  played = []
  for party in partys:
    played.append(predict_results_full(party[0], party[1], model_winner, model, clases))
    print(played[-1])
    match.append(predict_results(party[0], party[1], model_winner, model, clases))
  
  positions = dict(zip(Groups[group_X], [{"win":0.0,"GF":0.0, "DG":0.0}, 
                                         {"win":0.0,"GF":0.0, "DG":0.0}, 
                                         {"win":0.0,"GF":0.0, "DG":0.0}, 
                                         {"win":0.0,"GF":0.0, "DG":0.0}]))
  print()
  for i in range(len(match)):

    print(match[i])
    replys = match[i].split("_")
    cad = played[i].split("-")
    team_1 = int(cad[0][-1])
    team_2 = int(cad[1][0])

    if replys[0]==cad[0][:len(cad[0])-3]:
      if team_1>team_2:
        positions[replys[0]]["win"] += 1
      elif team_1<team_2:
        positions[replys[1]]["win"] += 1
      positions[replys[0]]["GF"] += team_1
      positions[replys[0]]["DG"] += team_1-team_2
      positions[replys[1]]["GF"] += team_2
      positions[replys[1]]["DG"] += team_2-team_1
    else:
      if team_2>team_1:
        positions[replys[0]]["win"] += 1
      elif team_1<team_2:
        positions[replys[1]]["win"] += 1
      positions[replys[0]]["GF"] += team_2
      positions[replys[0]]["DG"] += team_2-team_1
      positions[replys[1]]["GF"] += team_1
      positions[replys[1]]["DG"] += team_1-team_2   

  print()
  for i in sorted(positions, key=lambda x: positions[x]["win"]+(positions[x]["GF"]/100)+(positions[x]["DG"]/10000), reverse=True):
    print(i, positions[i])
  return sorted(positions, key=lambda x: positions[x]["win"]+(positions[x]["GF"]/100)+(positions[x]["DG"]/10000), reverse=True)[:2]

In [86]:
octavos = {}
for group in Groups:
  reply = win_groups(group)
  print("\n",reply,"\n\n","-"*50)
  octavos[group] = reply

======================> Group A
Qatar (1-2) Ecuador
Qatar (0-3) Senegal
Qatar (2-2) Netherlands
Ecuador (1-1) Senegal
Ecuador (0-8) Netherlands
Senegal (1-1) Netherlands

Ecuador_Qatar_+1
Senegal_Qatar_+3
Netherlands_Qatar_+0
Senegal_Ecuador_+0
Netherlands_Ecuador_+8
Netherlands_Senegal_+0

Netherlands {'win': 1.0, 'GF': 11.0, 'DG': 8.0}
Senegal {'win': 1.0, 'GF': 5.0, 'DG': 3.0}
Ecuador {'win': 1.0, 'GF': 3.0, 'DG': -7.0}
Qatar {'win': 0.0, 'GF': 3.0, 'DG': -4.0}

 ['Netherlands', 'Senegal'] 

 --------------------------------------------------
======================> Group B
England (1-0) Iran
England (1-0) United States
England (3-0) Wales
Iran (1-1) United States
Iran (0-1) Wales
United States (1-0) Wales

England_Iran_+1
England_United States_+1
England_Wales_+3
United States_Iran_+0
Wales_Iran_+1
United States_Wales_+1

England {'win': 3.0, 'GF': 5.0, 'DG': 5.0}
United States {'win': 1.0, 'GF': 2.0, 'DG': 0.0}
Wales {'win': 1.0, 'GF': 1.0, 'DG': -3.0}
Iran {'win': 0.0, 'GF': 1.0,

#Octavos

In [87]:
octavos

{'Group A': ['Netherlands', 'Senegal'],
 'Group B': ['England', 'United States'],
 'Group C': ['Argentina', 'Mexico'],
 'Group D': ['Denmark', 'Tunisia'],
 'Group E': ['Germany', 'Japan'],
 'Group F': ['Belgium', 'Morocco'],
 'Group G': ['Brazil', 'Serbia'],
 'Group H': ['Uruguay', 'South Korea']}

In [88]:
oct = list(octavos.keys())
cuartos_replys = []

for i in range(0, len(oct),2):
  cuartos_replys.append(predict_results(octavos[oct[i]][0], octavos[oct[i+1]][1], model_winner, model, clases, tie=False))
  print(predict_results_full(octavos[oct[i]][0], octavos[oct[i+1]][1], model_winner, model, clases, tie=False), end=" | ")
  print(cuartos_replys[-1])


for i in range(0, len(oct),2):
  cuartos_replys.append(predict_results(octavos[oct[i]][1], octavos[oct[i+1]][0], model_winner, model, clases, tie=False))
  print(predict_results_full(octavos[oct[i]][1], octavos[oct[i+1]][0], model_winner, model, clases, tie=False), end= " | ")
  print(cuartos_replys[-1])

Netherlands (2-0) United States | Netherlands_United States_+2
Argentina (4-0) Tunisia | Argentina_Tunisia_+4
Germany (3-1) Morocco | Germany_Morocco_+2
Brazil (3-0) South Korea | Brazil_South Korea_+3
Senegal (2-1) England | Senegal_England_+1
Mexico (3-1) Denmark | Mexico_Denmark_+2
Japan (2-3) Belgium | Belgium_Japan_+1
Serbia (0-2) Uruguay | Uruguay_Serbia_+2


# Cuartos

In [89]:
cuartos = [line.split("_")[0] for line in cuartos_replys]
cuartos

['Netherlands',
 'Argentina',
 'Germany',
 'Brazil',
 'Senegal',
 'Mexico',
 'Belgium',
 'Uruguay']

In [90]:
segundos_replys = []

for i in range(0, len(cuartos),2):
  print(predict_results_full(cuartos[i], cuartos[i+1], model_winner, model, clases, tie=False), end = " | ")
  segundos_replys.append(predict_results(cuartos[i], cuartos[i+1], model_winner, model, clases, tie=False))
  print(segundos_replys[-1])

Netherlands (4-0) Argentina | Netherlands_Argentina_+4
Germany (0-3) Brazil | Brazil_Germany_+3
Senegal (0-1) Mexico | Mexico_Senegal_+1
Belgium (3-0) Uruguay | Belgium_Uruguay_+3


# Semi-final & Final

In [91]:
segundos = [line.split("_")[0] for line in segundos_replys]
segundos

['Netherlands', 'Brazil', 'Mexico', 'Belgium']

In [92]:
semiF_replys = []

for i in range(0, len(segundos),2):
  semiF_replys.append(predict_results(segundos[i], segundos[i+1], model_winner, model, clases, tie=False))
  print(predict_results_full(segundos[i], segundos[i+1], model_winner, model, clases, tie=False), end=" | ")
  print(semiF_replys[-1])

pos4_and_3 = [line.split("_")[1] for line in semiF_replys]
tercero = predict_results(pos4_and_3[0], pos4_and_3[1], model_winner, model, clases, tie=False)
print()
print(predict_results_full(pos4_and_3[0], pos4_and_3[1], model_winner, model, clases, tie=False), end=" | ")
print(tercero)
tercero = tercero.split("_")[0]
pos4_and_3.remove(tercero)

print("\nFourth in the world cup:", pos4_and_3[0])
print("Third in the world cup:", tercero)

semiF = [line.split("_")[0] for line in semiF_replys]
print("\nThe Final is between:",semiF)

Netherlands (3-1) Brazil | Netherlands_Brazil_+2
Mexico (0-4) Belgium | Belgium_Mexico_+4

Brazil (5-1) Mexico | Brazil_Mexico_+4

Fourth in the world cup: Mexico
Third in the world cup: Brazil

The Final is between: ['Netherlands', 'Belgium']


In [93]:
# Final:

final = predict_results(semiF[0], semiF[1], model_winner, model, clases, tie=False)
print(predict_results_full(semiF[0], semiF[1], model_winner, model, clases, tie=False), end=" | ")
print(final)

semiF.remove(final.split("_")[0])
print("\nSecond in the world cup:", semiF[0])
print("\nWorld Cup Winner is: ", final.split("_")[0])

Netherlands (1-0) Belgium | Netherlands_Belgium_+1

Second in the world cup: Belgium

World Cup Winner is:  Netherlands
